In [1]:
import os
import sys
this = os.getcwd()
path = this[:this.rfind("/")]
if not path in sys.path: sys.path.append(path)
xgboost_path = path + "/xgboost_regressor"
promedios_path = path + "/promedio_zona"

import pandas as pd
import matplotlib
from datos import FEATURES_DISPONIBLES
from modelo import Modelo

pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
pd.set_option('display.max_columns', 1200)


In [3]:
from xgboost_regressor.xgboost_predictor import XGBoostRegressor
from promedio_zona.promedio_zona import PromedioZona
from regresion_lineal.regresion_lineal import RegresionLineal

In [4]:
class EnsambleConcatenados(XGBoostRegressor):
    """
        Usa el resultado de las predicciones del modelo PromedioZona
        para entrenar y predecir con un XGBoostRegressor.
    """
    
    
    @Modelo.cronometrar()
    def __init__(self):
        self.modelo_promedios = PromedioZona()
        self.modelo_lineal = RegresionLineal()
        super().__init__()        
        
    @Modelo.cronometrar()
    def cargar_datos(self):
        self.modelo_promedios.cargar_datos()
        self.modelo_lineal.cargar_datos()
        super().cargar_datos()
    
    @Modelo.cronometrar()
    def entrenar(self):
        self.agregar_predicciones_modelo(self.modelo_lineal)
        self.agregar_predicciones_modelo(self.modelo_promedios)
        super().entrenar()
    
    def agregar_predicciones_modelo(self, modelo):
        columna = 'prediccion_' + modelo.modelo
        modelo.entrenar()
        predicciones_train =  modelo.predecir(modelo.train_data)
        predicciones_test = modelo.predecir(modelo.test_data)
        predicciones_submit = modelo.predecir(modelo.submit_data)
        self.train_data[columna] = predicciones_train['target']
        self.test_data[columna] = predicciones_test['target']
        self.submit_data[columna] = predicciones_submit['target']


In [5]:
ensamble = EnsambleConcatenados()

instanciar demoro 0.0 segundos
instanciar demoro 0.0 segundos
instanciar demoro 0.0 segundos
__init__ demoro 0.0 segundos


In [6]:
ensamble.cargar_datos()

cargar demoro 1.39 segundos
cargar demoro 62.31 segundos
cargar demoro 63.31 segundos
cargar_datos demoro 130.18 segundos


In [7]:
ensamble.entrenar()

entrenar demoro 0.0 segundos
entrenar demoro 0.94 segundos
predecir demoro 0.14 segundos
predecir demoro 0.1 segundos
predecir demoro 0.1 segundos
entrenar demoro 0.0 segundos
entrenar demoro 0.21 segundos
predecir demoro 20.37 segundos
predecir demoro 6.77 segundos
predecir demoro 6.82 segundos
entrenar demoro 0.0 segundos
entrenar demoro 1360.3 segundos
entrenar demoro 1395.76 segundos


In [11]:
predicciones = ensamble.predecir(ensamble.test_data)

predecir demoro 3.53 segundos


In [12]:
columnas_predictoras = ['target', 'prediccion_PromedioZona', 'prediccion_RegresionLineal']
for columna in columnas_predictoras:
    predicciones['diferencia_'+columna] = predicciones['precio'] - predicciones[columna]

In [13]:
mejores_100 = predicciones.sort_values(by='diferencia_target').head(200)

In [14]:
peores_100 = predicciones.sort_values(by='diferencia_target').tail(200)

In [17]:
peores_100.describe()

,habitaciones,cantidad_palabras_titulo,precio,palabras_negativas_descripcion,palabras_positivas_descripcion,metrostotales,dia,banos,cantidad_palabras_descripcion,metroscubiertos,garages,ano,antiguedad,mes,dolar,prediccion_RegresionLineal,prediccion_PromedioZona,target,diferencia_target,diferencia_prediccion_PromedioZona,diferencia_prediccion_RegresionLineal
count,116.000,200.000,200.000,200.000,200.000,110.000,200.000,123.000,200.000,153.000,122.000,200.000,149.000,200.000,200.000,200.000,200.000,200.000,200.000,200.000,200.000
mean,3.586,4.330,10337906.000,0.210,1.650,239.750,15.240,2.674,31.275,254.375,0.902,2014.745,13.375,7.575,16.125,2814029.016,4171859.819,4153503.250,6184402.500,6166045.941,7523876.744
std,1.527,1.881,1616394.000,0.487,1.974,116.062,8.949,1.004,24.094,113.188,1.131,1.338,15.086,3.368,2.598,1807594.969,2694057.982,1794156.750,1379569.375,2370113.074,2007376.136
min,1.000,0.000,6000000.000,0.000,0.000,15.000,1.000,1.000,0.000,15.000,0.000,2012.000,0.000,1.000,12.492,556451.953,164868.078,404360.812,4807846.000,-1579789.375,529954.826
25%,3.000,3.000,9000000.000,0.000,0.000,152.000,7.000,2.000,14.750,180.000,0.000,2014.000,2.000,5.000,13.297,1587916.110,2368959.080,2692524.188,5229428.500,4889341.617,6289449.409
50%,3.000,4.000,10500000.000,0.000,1.000,245.000,15.000,3.000,26.000,260.000,0.000,2015.000,8.000,8.000,15.539,2272277.405,3954779.020,4272787.750,5748595.500,6222120.107,7667073.297
75%,4.000,5.250,11762500.000,0.000,2.000,316.750,23.000,4.000,39.000,340.000,2.000,2016.000,20.000,11.000,18.609,3617106.084,5612710.276,5481725.125,6651999.000,7586506.879,8929207.502
max,10.000,11.000,12520000.000,2.000,10.000,438.000,31.000,4.000,173.000,435.000,3.000,2016.000,76.000,12.000,19.750,11960123.849,13600101.562,7471721.500,11941317.000,12068199.182,11789226.047


In [16]:
mejores_100.describe()

,habitaciones,cantidad_palabras_titulo,precio,palabras_negativas_descripcion,palabras_positivas_descripcion,metrostotales,dia,banos,cantidad_palabras_descripcion,metroscubiertos,garages,ano,antiguedad,mes,dolar,prediccion_RegresionLineal,prediccion_PromedioZona,target,diferencia_target,diferencia_prediccion_PromedioZona,diferencia_prediccion_RegresionLineal
count,157.000,200.000,200.000,200.000,200.000,136.000,200.000,143.000,200.000,160.000,134.000,200.000,159.000,200.000,200.000,200.000,200.000,200.000,200.000,200.000,200.000
mean,3.389,4.355,2496175.250,0.245,1.995,289.500,15.800,2.686,34.440,293.500,1.590,2015.020,10.242,8.075,16.797,3525747.809,8265786.896,6657956.000,-4161780.250,-5769611.406,-1029572.319
std,1.207,2.081,1620940.250,0.516,1.994,inf,8.533,0.826,22.549,inf,1.171,1.264,10.070,3.661,2.654,1822835.197,10211179.012,1685423.000,870944.562,10180855.726,1901157.165
min,1.000,0.000,330000.000,0.000,0.000,18.000,1.000,1.000,0.000,20.000,0.000,2012.000,0.000,1.000,12.680,885520.164,120831.483,3930790.250,-9143918.000,-144363461.204,-8265737.153
25%,3.000,3.000,1200000.000,0.000,0.000,218.250,8.000,2.000,17.000,230.000,0.000,2014.000,3.000,5.000,13.523,2301983.703,5752197.361,5312867.250,-4366772.375,-6493515.599,-2187583.923
50%,3.000,4.000,2200000.000,0.000,1.000,300.000,16.000,3.000,32.000,295.000,2.000,2016.000,10.000,9.000,17.828,3130041.759,7569888.484,6553326.750,-3835861.750,-4777163.157,-929098.257
75%,4.000,6.000,3500000.000,0.000,3.000,358.500,23.000,3.000,45.000,357.750,3.000,2016.000,17.500,12.000,19.359,4509738.886,9447394.444,7684773.625,-3574010.125,-3409014.050,294532.355
max,10.000,10.000,6900000.000,3.000,9.000,435.000,31.000,4.000,111.000,439.000,3.000,2016.000,48.000,12.000,19.750,13265737.153,146363461.204,10531056.000,-3360039.500,613844.922,4114138.351


In [8]:
ensamble.validar()

predecir demoro 3.48 segundos
puntuar demoro 0.0 segundos
validar demoro 3.48 segundos


534928.4

In [9]:
predicciones = ensamble.predecir(ensamble.submit_data)

predecir demoro 3.53 segundos


In [10]:
comentario = "con mejoras en modelo lineal y por zonas - local 534928.4"
ensamble.presentar(predicciones, comentario)

guardar demoro 0.13 segundos


100%|██████████| 967k/967k [00:07<00:00, 126kB/s]  


submit demoro 10.01 segundos
presentar demoro 10.14 segundos
